### Authenticated key exchange with DH and DSA

In [1]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import dh, dsa
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

In [2]:
parameters = dh.generate_parameters(generator=2, key_size=2048)
alice_dh_private_key = parameters.generate_private_key()
alice_dh_public_key = alice_dh_private_key.public_key()

bob_dh_private_key = parameters.generate_private_key()
bob_dh_public_key = bob_dh_private_key.public_key()

print("public and private keys for alice and bob have been generated")


public and private keys for alice and bob have been generated


In [3]:
alice_dsa_private_key = dsa.generate_private_key(key_size=2048)
alice_dsa_public_key = alice_dsa_private_key.public_key()
bob_dsa_private_key = dsa.generate_private_key(key_size=2048)
bob_dsa_public_key = bob_dsa_private_key.public_key()

print("additional pairs for signing generated!")

additional pairs for signing generated!


Now, alice will sign her dh public key using dsa private key

In [5]:
alice_public_bytes = alice_dh_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

alice_signature = alice_dsa_private_key.sign(alice_public_bytes, hashes.SHA256())
print("alice signed her public key")

alice signed her public key


In [6]:
bob_public_bytes = bob_dh_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo)
bob_signature = bob_dsa_private_key.sign(bob_public_bytes, hashes.SHA256())

print("Bob signed public key")

Bob signed public key


Now, verify if the signatures are valid

In [8]:
alice_dsa_public_key.verify(alice_signature, alice_public_bytes, hashes.SHA256())

bob_dsa_public_key.verify(bob_signature, bob_public_bytes, hashes.SHA256())

print("yaya valid signatures")

yaya valid signatures


In [10]:
alice_shared_key = alice_dh_private_key.exchange(bob_dh_public_key)
bob_shared_key = bob_dh_private_key.exchange(alice_dh_public_key)

print("shared secrets have been generated")

shared secrets have been generated


Optionally, we can use a specialized kdf to generate a more secture symmetric key

In [12]:
def derive_key(shared_key):
    return HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=None
    ).derive(shared_key)

alice_symmetric_key = derive_key(alice_shared_key)
bob_symmetric_key = derive_key(bob_shared_key)

assert(alice_symmetric_key == bob_symmetric_key)
print("keys same")

keys same
